In [1]:
import pandas as pd

train = pd.read_csv('../../data/train.csv')
test = pd.read_csv('../../data/test.csv')

train.head()

,ID,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,y
0,TRAIN_00000,1.006187,-1.962566,1.247535,0.926500,-0.265766,-1.789301,0.470004,-0.139467,0.623996,0.320359,0.078612,83.424500
1,TRAIN_00001,1.024647,-2.472625,1.144386,0.846499,-0.287336,-1.756679,0.503860,-0.219545,0.697607,0.238306,0.081778,79.374109
2,TRAIN_00002,1.062444,-2.451003,1.186546,0.873599,-0.257828,-1.802735,0.498045,-0.194247,0.684134,0.259392,0.095003,82.181616
3,TRAIN_00003,1.089189,-2.458470,1.184531,0.810867,-0.276517,-1.787739,0.503359,-0.201923,0.686394,0.245736,0.091737,83.006586
4,TRAIN_00004,1.023323,-2.133468,1.242266,0.939837,-0.264515,-1.792044,0.470478,-0.142896,0.623778,0.314610,0.078987,83.051434


In [2]:
train_X = train.drop(columns=['ID','y'])
train_y = train['y']

train_data = pd.concat([train_X, train_y], axis=1)

In [3]:
# normalization
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

train_X_nor = minmax.fit_transform(train_X)

train_X_nor = pd.DataFrame(train_X_nor, columns=train_X.columns)

train_data_nor = pd.concat([train_X_nor, train_y.reset_index(drop=True)], axis=1) # 인덱스 불일치로 인한 결합 오류 방지

In [4]:
# standardization
from sklearn.preprocessing import StandardScaler

standard = StandardScaler()

train_X_std = standard.fit_transform(train_X)

train_X_std = pd.DataFrame(train_X_std, columns=train_X.columns)

train_data_std = pd.concat([train_X_std, train_y.reset_index(drop=True)], axis=1) # 인덱스 불일치로 인한 결합 오류 방지

In [5]:
from pycaret.regression import setup, compare_models, pull, save_model, load_model, predict_model

regressor = setup(data = train_data_std, target='y', session_id=42)
best_model = compare_models()
results = pull()

results

,Description,Value
0,Session id,42
1,Target,y
2,Target type,Regression
3,Original data shape,"(40118, 12)"
4,Transformed data shape,"(40118, 12)"
5,Transformed train set shape,"(28082, 12)"
6,Transformed test set shape,"(12036, 12)"
7,Numeric features,11
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,1.2057,2.8778,1.6879,0.6054,0.0220,0.0146,0.0240
lr,Linear Regression,1.2057,2.8779,1.6879,0.6053,0.0220,0.0146,0.4660
ridge,Ridge Regression,1.2057,2.8779,1.6879,0.6053,0.0220,0.0146,0.0100
gbr,Gradient Boosting Regressor,1.2084,2.8859,1.6903,0.6042,0.0220,0.0146,2.3850
lightgbm,Light Gradient Boosting Machine,1.2119,2.8978,1.6939,0.6026,0.0220,0.0147,0.4140
huber,Huber Regressor,1.1968,2.9038,1.6958,0.6017,0.0221,0.0145,0.0800
et,Extra Trees Regressor,1.2374,2.9841,1.7193,0.5905,0.0223,0.0150,1.3440
rf,Random Forest Regressor,1.2356,2.9932,1.7219,0.5894,0.0224,0.0150,5.6150
omp,Orthogonal Matching Pursuit,1.3052,3.2432,1.7936,0.5546,0.0231,0.0158,0.0270
en,Elastic Net,1.2696,3.4312,1.8461,0.5283,0.0236,0.0153,0.0080


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,1.2057,2.8778,1.6879,0.6054,0.0220,0.0146,0.024
lr,Linear Regression,1.2057,2.8779,1.6879,0.6053,0.0220,0.0146,0.466
ridge,Ridge Regression,1.2057,2.8779,1.6879,0.6053,0.0220,0.0146,0.010
gbr,Gradient Boosting Regressor,1.2084,2.8859,1.6903,0.6042,0.0220,0.0146,2.385
lightgbm,Light Gradient Boosting Machine,1.2119,2.8978,1.6939,0.6026,0.0220,0.0147,0.414
huber,Huber Regressor,1.1968,2.9038,1.6958,0.6017,0.0221,0.0145,0.080
et,Extra Trees Regressor,1.2374,2.9841,1.7193,0.5905,0.0223,0.0150,1.344
rf,Random Forest Regressor,1.2356,2.9932,1.7219,0.5894,0.0224,0.0150,5.615
omp,Orthogonal Matching Pursuit,1.3052,3.2432,1.7936,0.5546,0.0231,0.0158,0.027
en,Elastic Net,1.2696,3.4312,1.8461,0.5283,0.0236,0.0153,0.008


In [6]:
save_model(best_model, 'best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['x_0', 'x_1', 'x_2', 'x_3', 'x_4',
                                              'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
                                              'x_10'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model', BayesianRidge())]),
 'best_model.pkl')

In [7]:
test_X = test.drop(columns=['ID'])
best_model = load_model('best_model')
preds = predict_model(best_model, data = test_X)

preds

Transformation Pipeline and Model Successfully Loaded


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,prediction_label
0,1.069219,-2.106844,1.228438,0.897236,-0.258323,-1.794707,0.467295,-0.154325,0.630702,0.312034,0.084618,84.657227
1,1.071553,-2.509419,1.153271,0.824917,-0.270175,-1.773425,0.501530,-0.203479,0.693976,0.246928,0.086857,84.840927
2,0.977364,-1.362481,1.164106,0.948528,-0.417688,-1.633350,0.517789,-0.081149,0.581488,0.386941,0.002027,84.410934
3,0.977220,-1.211227,1.138849,0.914028,-0.459511,-1.602302,0.538293,-0.069170,0.580191,0.397438,-0.017518,84.347488
4,1.069111,-2.409491,1.174731,0.856938,-0.273046,-1.782787,0.504689,-0.197011,0.690465,0.246498,0.089634,84.778656
...,...,...,...,...,...,...,...,...,...,...,...,...
4981,1.037452,-2.132522,1.236089,0.860538,-0.252378,-1.805769,0.472018,-0.158855,0.636707,0.302851,0.087101,84.651733
4982,0.932432,-1.193839,1.139827,0.955182,-0.471994,-1.600613,0.541196,-0.071568,0.583131,0.397209,-0.017859,84.340553
4983,1.083483,-2.371390,1.214140,0.850023,-0.247225,-1.811753,0.481727,-0.175863,0.661456,0.287690,0.094479,84.784210
4984,1.072450,-2.377752,1.234282,0.882019,-0.251468,-1.809643,0.479923,-0.168748,0.667690,0.278757,0.096796,84.799469


In [8]:
submission = pd.read_csv('../../data/sample_submission.csv')
submission['y'] = preds['prediction_label']
submission.to_csv('../normalized_AutoML/std_AutoML_submission.csv',index=False)